In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 100)

import pandas as pd
import numpy as np
import os
import seaborn as sns
from ipywidgets import interact, fixed
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import animation
from matplotlib.animation import FFMpegWriter
pd.set_option('max_columns', 100)

import dateutil
from math import radians
from IPython.display import Video

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
from sklearn.neighbors import BallTree
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from datetime import datetime
import pytz
from IPython.display import HTML
import scipy.stats as stats
import matplotlib as mpl
from matplotlib import animation, rc
from matplotlib.patches import Rectangle, Arrow
import tensorflow as tf

#from BDBUtils.Utilities import CreateNFLData
from IPython.core.display import HTML
import time
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 1000)
from tqdm import tqdm
from sklearn.neighbors import BallTree
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from datetime import datetime
import pytz
from IPython.display import HTML
import scipy.stats as stats
import matplotlib as mpl
from matplotlib import animation, rc
from matplotlib.patches import Rectangle, Arrow
import tensorflow as tf


def reduce_mem_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def get_dx_dy(radian_angle, dist):
    dx = dist * math.cos(radian_angle)
    dy = dist * math.sin(radian_angle)
    return dx, dy



class CreateNFLData:

    def __init__(self):
        pass

    def LoadData(self, Normal=True):
        if Normal == True:
            print("Loading Original Data")
            globbed_files = glob.glob("../input/nfl-big-data-bowl-2021/week*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        
        else:
            print("Loading Modified Data")
            globbed_files = glob.glob("Revised Data/*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        return WeekData



    def Standardize(self,W):
        print("Standardizing Data..")
        W['Dir_rad'] = np.mod(90 - W.dir, 360) * math.pi/180.0
        W['ToLeft'] = W.playDirection == "left"
        W['TeamOnOffense'] = "home"
        W.loc[W.possessionTeam != W.PlayerTeam, 'TeamOnOffense'] = "away"
        W['IsOnOffense'] = W.PlayerTeam == W.TeamOnOffense # Is player on offense?
        W['YardLine_std'] = 100 - W.yardlineNumber
        W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'YardLine_std'
                ] = W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'yardlineNumber']
        W['X_std'] = W.x
        W.loc[W.ToLeft, 'X_std'] = 120 - W.loc[W.ToLeft, 'x'] 
        W['Y_std'] = W.y
        W.loc[W.ToLeft, 'Y_std'] = 160/3 - W.loc[W.ToLeft, 'y'] 
        #W['Orientation_std'] = -90 + W.Orientation
        #W.loc[W.ToLeft, 'Orientation_std'] = np.mod(180 + W.loc[W.ToLeft, 'Orientation_std'], 360)
        W['Dir_std'] = W.Dir_rad
        W.loc[W.ToLeft, 'Dir_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Dir_rad'], 2*np.pi)
        W['dx'] = round(W['s']*np.cos(W['Dir_std']),2)
        W['dy'] = round(W['s']*np.sin(W['Dir_std']),2)
        W['X_std'] = round(W['X_std'],2)
        W['Y_std'] = round(W['Y_std'],2)
        #W['Orientation_rad'] = np.mod(W.o, 360) * math.pi/180.0
        W['Orientation_rad'] = np.mod(-W.o + 90, 360) * math.pi/180.0
        W['Orientation_std'] = W.Orientation_rad
        W.loc[W.ToLeft, 'Orientation_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Orientation_rad'], 2*np.pi)
        W['MPH'] = W['s'] / 0.488889
        return W


    def mergeGameData2(self,df):
        print("Merging Data..")
        games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv', index_col=0)
        plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv', index_col=0, usecols= ['gameId', 'playId','yardlineNumber','possessionTeam','yardlineSide'])
        df = pd.merge(df, games, how='left', left_on=['gameId'], right_on=['gameId'])
        df['PlayerTeam'] = np.where(df['team'] == 'home', df['homeTeamAbbr'], df['visitorTeamAbbr'] )
        df = df.drop(['homeTeamAbbr','visitorTeamAbbr'], axis=1)
        df = pd.merge(df, plays, how='left', left_on=['gameId', 'playId'], right_on=['gameId', 'playId'])
        df['OnOffense'] = np.where(df['PlayerTeam'] == df['possessionTeam'], True, False)
        TargetedWR = pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/targetedReceiver.csv', index_col=0)
        Coverages = pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/coverages_week1.csv', index_col=0)
        df = pd.merge(df, Coverages, how='left', left_on=['gameId','playId'], right_on=['gameId','playId'])
        df = pd.merge(df, TargetedWR, how='left', left_on=['gameId','playId'], right_on=['gameId','playId'])
        Offense = df.query('OnOffense == 1').reset_index()
        Defense = df.query('OnOffense == 0').reset_index()
        Football = df.query('displayName == "Football"').reset_index()
        return df
    
    def PositionRank(self,df):
        print("Creating Position Ranks..")
        group = df.groupby(['week','gameId','possessionTeam','targetNflId','playId'])['offensePlayResult','epa_x','yac_epa','air_epa','air_yards','complete_pass','touchdown'].max().reset_index()
        group = group.groupby(['week','gameId','possessionTeam','targetNflId'])['offensePlayResult','epa_x','yac_epa','air_epa','air_yards','complete_pass','touchdown'].agg(['sum','count']).reset_index()
        group.columns = [' '.join(col).strip() for col in group.columns.values]
        group.columns = group.columns.str.replace('sum' , '')
        #wrs= df.filter(['gameId','targetNflId',"receiver_player_id","complete_pass","down","qb_dropback","air_yards","yards_after_catch",'yards_gained','shotgun','no_huddle','pass_touchdown','yardline_100','ydstogo','pass_length','pass_location','epa','air_epa','yac_epa'])
        #wrs['yards_after_catch'] = wrs['yards_after_catch'].fillna(0)

        #wrs['Targets'] = (wrs['yardline_100'] > 0).astype(int)

        Cols = ['week', 'gameId','possessionTeam','targetNflId','offensePlayResult count','complete_pass ','touchdown ', 'offensePlayResult ', 'air_yards ']
        group = group[Cols]
        group.columns = ['week', 'gameId','Team','targetNflId','Targets','Rec','TD', 'Yards ', 'Air_yards']

        group = group.drop_duplicates(subset=['week','gameId','targetNflId'])
        Numdf = ['Targets','Rec','TD', 'Yards ', 'Air_yards']
        for col in tqdm(Numdf):
            group[col + '_Rolling'] = group.groupby(['targetNflId']).apply(lambda x: x[col].shift(1).expanding(1).mean()).reset_index(0,drop=True)

        group['Targets_Rolling'] = group['Targets_Rolling'].fillna(group.groupby(['targetNflId'])['Targets_Rolling'].transform('median'))
        group['Rec_Rolling'] = group['Rec_Rolling'].fillna(group.groupby(['targetNflId'])['Rec_Rolling'].transform('median'))
        group['TD_Rolling'] = group['TD_Rolling'].fillna(group.groupby(['targetNflId'])['TD_Rolling'].transform('median'))
        group['Yards _Rolling'] = group['Yards _Rolling'].fillna(group.groupby(['targetNflId'])['Yards _Rolling'].transform('median'))
        group['Air_yards_Rolling'] = group['Air_yards_Rolling'].fillna(group.groupby(['targetNflId'])['Air_yards_Rolling'].transform('median'))
        players = pd.read_csv('players.csv', index_col=0, usecols=['nflId','position','displayName'])
        group = pd.merge(group, players, how='left',left_on=['targetNflId'],right_on=['nflId'])
        group['Pos_Score'] = group['Targets_Rolling']*.35	 + group['Yards _Rolling']*.30 + group['Air_yards_Rolling']*.20 + group['TD_Rolling']*.15
        group['Pos_Score'] = group['Pos_Score'].fillna(0)
        group['Pos_Rank'] = group['position'] + round(group.groupby(['Team','gameId','position'])['Pos_Score'].rank(ascending=False),0).astype(int).astype(str)

        Cols = ['gameId','targetNflId','Pos_Rank']
        group1 = group[Cols]


        Cols = [1,3,5,7,9,13]

        for c in Cols:
            trim1 = group[(group['Pos_Rank'] == "WR1") & (group['week'] == c) ].filter(['displayName','Team','Pos_Score','Pos_Rank'],axis=1).sort_values(by=['Pos_Score'], ascending=False).reset_index().head(5)
            trim2 = group[(group['Pos_Rank'] == "WR2") & (group['week'] == c) ].filter(['displayName','Team','Pos_Score','Pos_Rank'],axis=1).sort_values(by=['Pos_Score'], ascending=False).reset_index().head(5)
            print('Week ',str(c))
            display(HTML(trim1.head(5).to_html()))
            display(HTML(trim2.head(5).to_html()))
        group1.to_csv('Pos_Rank.csv')
        df1 = pd.merge(df, group1, how='left',left_on=['gameId','nflId'],right_on=['gameId','targetNflId'])
        return df1
    
    def FrameData(self,df):
        NotNone = df.query('event != "None"')
        NotNone = NotNone.groupby(['gameId','playId','event'])['frameId'].max().reset_index()
        NotNone = NotNone.set_index(['gameId','playId','event'], drop= True).unstack('event').reset_index()
        NotNone.columns = [' '.join(col).strip() for col in NotNone.columns.values]
        NotNone.columns = NotNone.columns.str.replace('frameId' , '')
        NotNone.columns = NotNone.columns.str.replace(' ' , '')

        for col in tqdm(NotNone.columns):
            NotNone['Contains_' + str(col)] = np.where(NotNone[col] > 0, True, False)


        WeekData1 = pd.merge(df, NotNone, how="left", left_on=['gameId','playId'], right_on=['gameId','playId'] )
        WeekData1['After_snap'] = np.where(WeekData1['frameId'] > WeekData1['ball_snap'],1,0)
        WeekData1['After_Throw'] = np.where(WeekData1['frameId'] > WeekData1['pass_forward'],1,0)
        WeekData1['After_PassArrived'] = np.where(WeekData1['frameId'] > WeekData1['pass_arrived'],1,0)
        WeekData1['After_PlayAction'] = np.where(WeekData1['frameId'] > WeekData1['play_action'],1,0)
        WeekData1['After_run_pass_option'] = np.where(WeekData1['frameId'] > WeekData1['run_pass_option'],1,0)
        WeekData1['After_Catch'] = np.where(WeekData1['frameId'] > WeekData1['pass_outcome_caught'],1,0)
        return WeekData1
    
    def import_data(self,file):

        df = pd.read_csv(file)
        df = reduce_mem_usage(df)
        return df
    
    def GetRelativePlayers(self,weekMod):
        groupedWeek = weekMod.groupby(['gameId', 'playId', 'frameId'])
        playerXY = {}
        for name, group in tqdm(groupedWeek):
            playerXY[name] = []
            for row in group.iterrows():
                data = [row[1]['nflId'], row[1]['team'], row[1]['x'], row[1]['y'], row[1]['position']]
                playerXY[name].append(data)




        features = list(weekMod.columns)
        weekArray = np.array(weekMod)
        minOppDist = []
        for player in tqdm(weekArray):
            if player[features.index('team')] != 'football':
                try:
                    opponentPositions = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
                    distances = []
                    directions = []
                    opponents = []
                    xs = []
                    ys = []
                    for oppPos in opponentPositions: 
                        if player[features.index('team')] != oppPos[1] and player[features.index('team')] != 'football' and oppPos[1] != 'football':
                            dx = (player[features.index('x')] - oppPos[2])**2
                            dy = (player[features.index('y')] - oppPos[3])**2
                            dist = np.sqrt(dx+dy)
                            distances.append(dist)
                            opponents.append(oppPos[0])
                    minDist = min(distances)
                    closestOpponent = opponents[np.argmin(distances)]
                    summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestOpponent]
                    minOppDist.append(summary)
                except:
                    continue

        minMateDist = []
        for player in tqdm(weekArray):
            if player[features.index('team')] != 'football':
                try:
                    matePositions = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
                    distances = []
                    mates = []
                    xs = []
                    ys = []
                    for matePos in matePositions: 
                        if player[features.index('team')] == matePos[1] and player[features.index('nflId')] != matePos[0] and player[features.index('team')] != 'football' and matePos[1] != 'football':
                            dx = (player[features.index('x')] - matePos[2])**2
                            dy = (player[features.index('y')] - matePos[3])**2
                            dist = np.sqrt(dx+dy)
                            distances.append(dist)
                            mates.append(matePos[0])
                    minDist = min(distances)
                    closestTm = mates[np.argmin(distances)]
                    summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestTm]
                    minMateDist.append(summary)
                except:
                    continue

        minQBDist = []
        for player in tqdm(weekArray):
            if player[features.index('team')] != 'football':
                try:
                    QBs = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
                    distances = []
                    QBz = []
                    xs = []
                    ys = []
                    for QB in QBs:
                        try:
                            if QB[4] == "QB":
                                dx = (player[features.index('x')] - QB[2])**2
                                dy = (player[features.index('y')] - QB[3])**2
                                dist = np.sqrt(dx+dy)
                                distances.append(dist)
                                QBz.append(QB[0])
                        except:
                            continue
                    QBminDist = min(distances)
                    QB = QBz[np.argmin(distances)]
                    summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], QBminDist, QB]
                    minQBDist.append(summary)
                except:
                    continue

        minFootballDist = []
        for player in tqdm(weekArray):
            try:
                Footballs = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
                distances = []
                xs = []
                ys = []
                for Foot in Footballs:
                    try: 
                        if Foot[1] == "football":
                            dx = (player[features.index('x')] - Foot[2])**2
                            dy = (player[features.index('y')] - Foot[3])**2
                            dist = np.sqrt(dx+dy)
                            distances.append(dist)
                    except:
                        continue
                minDist = min(distances)
                summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist]
                minFootballDist.append(summary)
            except:
                continue

        minMateDist = pd.DataFrame(minMateDist, columns=['gameId', 'playId', 'frameId', 'nflId','Team_Dist', 'closestTeam_Id'])
        minOppDist = pd.DataFrame(minOppDist, columns=['gameId', 'playId', 'frameId', 'nflId','Opp_Dist', 'closestOpp_Id'])
        minQBDist = pd.DataFrame(minQBDist, columns=['gameId', 'playId', 'frameId', 'nflId','QB_Dist', 'closestQB_Id'])
        minFootDist = pd.DataFrame(minFootballDist, columns=['gameId', 'playId', 'frameId', 'nflId','FootDist'])
        minOppDist = pd.merge(minOppDist, minMateDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
        minOppDist = pd.merge(minOppDist, minQBDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
        minOppDist = pd.merge(minOppDist, minFootDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
        return minOppDist
        
    

    





In [ ]:
def GetRelativePlayers(weekMod):
    groupedWeek = weekMod.groupby(['gameId', 'playId', 'frameId'])
    playerXY = {}
    for name, group in tqdm(groupedWeek):
        playerXY[name] = []
        for row in group.iterrows():
            data = [row[1]['nflId'], row[1]['team'], row[1]['x'], row[1]['y'], row[1]['position']]
            playerXY[name].append(data)




    features = list(weekMod.columns)
    weekArray = np.array(weekMod)
    minOppDist = []
    for player in tqdm(weekArray):
        if player[features.index('team')] != 'football':
            try:
                opponentPositions = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
                distances = []
                directions = []
                opponents = []
                xs = []
                ys = []
                for oppPos in opponentPositions: 
                    if player[features.index('team')] != oppPos[1] and player[features.index('team')] != 'football' and oppPos[1] != 'football':
                        dx = (player[features.index('x')] - oppPos[2])**2
                        dy = (player[features.index('y')] - oppPos[3])**2
                        dist = np.sqrt(dx+dy)
                        distances.append(dist)
                        opponents.append(oppPos[0])
                minDist = min(distances)
                closestOpponent = opponents[np.argmin(distances)]
                summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestOpponent]
                minOppDist.append(summary)
            except:
                continue

    minMateDist = []
    for player in tqdm(weekArray):
        if player[features.index('team')] != 'football':
            try:
                matePositions = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
                distances = []
                mates = []
                xs = []
                ys = []
                for matePos in matePositions: 
                    if player[features.index('team')] == matePos[1] and player[features.index('nflId')] != matePos[0] and player[features.index('team')] != 'football' and matePos[1] != 'football':
                        dx = (player[features.index('x')] - matePos[2])**2
                        dy = (player[features.index('y')] - matePos[3])**2
                        dist = np.sqrt(dx+dy)
                        distances.append(dist)
                        mates.append(matePos[0])
                minDist = min(distances)
                closestTm = mates[np.argmin(distances)]
                summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestTm]
                minMateDist.append(summary)
            except:
                continue

    minQBDist = []
    for player in tqdm(weekArray):
        if player[features.index('team')] != 'football':
            try:
                QBs = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
                distances = []
                QBz = []
                xs = []
                ys = []
                for QB in QBs:
                    try:
                        if QB[4] == "QB":
                            dx = (player[features.index('x')] - QB[2])**2
                            dy = (player[features.index('y')] - QB[3])**2
                            dist = np.sqrt(dx+dy)
                            distances.append(dist)
                            QBz.append(QB[0])
                    except:
                        continue
                QBminDist = min(distances)
                QB = QBz[np.argmin(distances)]
                summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], QBminDist, QB]
                minQBDist.append(summary)
            except:
                continue

    minFootballDist = []
    for player in tqdm(weekArray):
        try:
            Footballs = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
            distances = []
            xs = []
            ys = []
            for Foot in Footballs:
                try: 
                    if Foot[1] == "football":
                        dx = (player[features.index('x')] - Foot[2])**2
                        dy = (player[features.index('y')] - Foot[3])**2
                        dist = np.sqrt(dx+dy)
                        distances.append(dist)
                except:
                    continue
            minDist = min(distances)
            summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist]
            minFootballDist.append(summary)
        except:
            continue

    minMateDist = pd.DataFrame(minMateDist, columns=['gameId', 'playId', 'frameId', 'nflId','Team_Dist', 'closestTeam_Id'])
    minOppDist = pd.DataFrame(minOppDist, columns=['gameId', 'playId', 'frameId', 'nflId','Opp_Dist', 'closestOpp_Id'])
    minQBDist = pd.DataFrame(minQBDist, columns=['gameId', 'playId', 'frameId', 'nflId','QB_Dist', 'closestQB_Id'])
    minFootDist = pd.DataFrame(minFootballDist, columns=['gameId', 'playId', 'frameId', 'nflId','FootDist'])
    minOppDist = pd.merge(minOppDist, minMateDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
    minOppDist = pd.merge(minOppDist, minQBDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
    minOppDist = pd.merge(minOppDist, minFootDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
    return minOppDist

In [ ]:
import gc

Create = CreateNFLData()

start = time.process_time()
Weeks = range(1,18)

for n in Weeks:
    print('Week ' + str(n))
    filename = '../input/nfl-big-data-bowl-2021/week' + str(n) + '.csv'
    Week = pd.read_csv(filename)
    Final = Create.GetRelativePlayers(Week)
    Final.to_csv('Relative_Info_Week ' + str(n) + ".csv")
    gc.collect()

print((time.process_time() - start) / 60)

In [ ]:
import gc

def mergeframes(Maindf,df):
    Finaldf1 = pd.merge(Maindf, df, how='left', left_on=['gameId','playId','frameId','nflId'], right_on=['gameId','playId','frameId','nflId_x'])
    return Finaldf1

start = time.process_time()

Pos_Rank = pd.read_csv('../input/position-rankings/Pos_Rank.csv', index_col=0)

Weeks = range(1,18)

for n in tqdm(Weeks):
    start = time.process_time()
    print('Week ' + str(n))
    filename = '../input/nfl-big-data-bowl-2021/week' + str(n) + '.csv'
    filename2 = 'Relative_Info_Week ' + str(n) + '.csv'
    Create = CreateNFLData()
    Original = Create.import_data(filename)
    df = Create.mergeGameData2(Original)
    Finaldf = Create.Standardize(df)
    PlayerData = Finaldf.filter(['gameId','playId','frameId','nflId','position','route','X_std','Y_std','Dir_std','dx','dy','Orientation_std','MPH','s','a','dis'], axis=1)
    PlayerData = pd.merge(PlayerData, Pos_Rank, how='left', left_on=['gameId','nflId'], right_on=['gameId','targetNflId'])
    PlayerData = PlayerData.drop(['targetNflId'], axis=1)

    Rev = Create.import_data(filename2)
    Opp = Rev.filter(['gameId','playId','frameId','nflId','closestOpp_Id','Opp_Dist'], axis=1)
    Tm = Rev.filter(['gameId','playId','frameId','nflId','closestTeam_Id','Team_Dist'], axis=1)
    Qb = Rev.filter(['gameId','playId','frameId','nflId','closestQB_Id','QB_Dist'], axis=1)
    foot = Rev.filter(['gameId','playId','frameId','nflId','FootDist'], axis=1)

    Opp = pd.merge(Opp,PlayerData,how='left',left_on=['gameId','playId','frameId','closestOpp_Id'], right_on=['gameId','playId','frameId','nflId'])
    keep_same = {'gameId','playId','frameId','nflId_x','closestOpp_Id'}
    Opp.columns = ['{}{}'.format(c, '' if c in keep_same else '_COpp') for c in Opp.columns]

    Tm = pd.merge(Tm,PlayerData,how='left',left_on=['gameId','playId','frameId','closestTeam_Id'], right_on=['gameId','playId','frameId','nflId'])
    keep_same = {'gameId','playId','frameId','nflId_x','closestTeam_Id'}
    Tm.columns = ['{}{}'.format(c, '' if c in keep_same else '_CTm') for c in Tm.columns]

    Qb = pd.merge(Qb,PlayerData,how='left',left_on=['gameId','playId','frameId','closestQB_Id'], right_on=['gameId','playId','frameId','nflId'])
    keep_same = {'gameId','playId','frameId','nflId_x','closestQB_Id'}
    Qb.columns = ['{}{}'.format(c, '' if c in keep_same else '_QB') for c in Qb.columns]
    Qb = Qb.drop(['Dir_std_QB','dx_QB','dy_QB','MPH_QB','s_QB','a_QB','dis_QB','position_QB'], axis = 1)

    Finaldf['Code'] = Finaldf['gameId'].astype(str) + "-" + Finaldf['playId'].astype(str) + "-" + Finaldf['frameId'].astype(str) + "-" + Finaldf['nflId'].astype(str)
    foot['Code'] = foot['gameId'].astype(str) + "-" + foot['playId'].astype(str) + "-" + foot['frameId'].astype(str) + "-" + foot['nflId'].astype(str)
    foot = foot.set_index('Code')
    foot = foot.loc[~foot.index.duplicated(keep='first')]

    foot = foot.drop(['gameId','playId','frameId','nflId'], axis=1)


    Finaldf4 = mergeframes(Finaldf, Opp)
    Finaldf4 = mergeframes(Finaldf4, Tm)
    Finaldf4 = mergeframes(Finaldf4, Qb)

    #   Finaldf1 = pd.merge(Finaldf, Opp, how='left', left_on=['gameId','playId','frameId','nflId'], right_on=['gameId','playId','frameId','nflId_x'])
    #   Finaldf2 = pd.merge(Finaldf1, Tm, how='left', left_on=['gameId','playId','frameId','nflId'], right_on=['gameId','playId','frameId','nflId_x'])
    #   Finaldf4 = pd.merge(Finaldf2, Qb, how='left', left_on=['gameId','playId','frameId','nflId'], right_on=['gameId','playId','frameId','nflId_x'])

    Finaldf4["FootDist"] = Finaldf4.Code.map(foot['FootDist'])


    Finaldf4['Targeted'] = np.where((Finaldf4['closestOpp_Id'] == Finaldf4['targetNflId']) | (Finaldf4['nflId'] == Finaldf4['targetNflId']), True, False)

    Finaldf4 = Finaldf4.drop(['nflId_x_x','nflId_x_y','nflId_y_QB','nflId_x','route_QB','Pos_Rank_QB','nflId_y_COpp'],axis=1)
    Cols = [ 'gameDate', 'gameTimeEastern', 'week', 'gameId','playId','frameId','time', 'nflId', 'displayName', 'jerseyNumber', 'position',  'team',  'X_std', 'Y_std', 'Dir_std', 'dx', 'dy', 'Orientation_std', 's', 'MPH', 'a', 'dis', 'event','playDirection', 'route','PlayerTeam','possessionTeam','yardlineSide', 'yardlineNumber', 'YardLine_std', 'OnOffense', 'coverage', 'targetNflId','ToLeft', 'TeamOnOffense', 'IsOnOffense','closestOpp_Id', 'Opp_Dist_COpp', 'Opp_Dist_COpp', 'route_COpp', 'X_std_COpp', 'Y_std_COpp', 'Dir_std_COpp', 'dx_COpp', 'dy_COpp', 'Orientation_std_COpp', 'MPH_COpp', 's_COpp', 'a_COpp', 'dis_COpp', 'position_COpp', 'Pos_Rank_COpp', 'closestTeam_Id', 'Team_Dist_CTm', 'nflId_y_CTm', 'route_CTm', 'X_std_CTm', 'Y_std_CTm', 'Dir_std_CTm', 'dx_CTm', 'dy_CTm', 'Orientation_std_CTm', 'MPH_CTm', 's_CTm', 'a_CTm', 'dis_CTm', 'position_CTm', 'Pos_Rank_CTm', 'closestQB_Id', 'QB_Dist_QB', 'X_std_QB', 'Y_std_QB', 'Orientation_std_QB', 'FootDist', 'Targeted']
    Finaldf4 = Finaldf4[Cols]
    print((time.process_time() - start) / 60)
    print(Finaldf4.shape)
    Finaldf4 = Finaldf4.drop_duplicates(subset=['gameId','playId','frameId','nflId'], keep='last')
    print(Finaldf4.shape)
    Finaldf4.to_csv('FinalData_Week' + str(n) + '.gz', compression='gzip', index=False)
    gc.collect()

print((time.process_time() - start) / 60)